In [2]:
# =====================================================
# ZleepAnlystNet - เวอร์ชันเข้าใจง่าย (ตาม paper 100%)
# ไม่ใช้ OOP ซับซ้อน ใช้ loop ธรรมดาแทน
# =====================================================
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import pandas as pd
from collections import Counter
import mne

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"ใช้ device: {device}")

ใช้ device: cpu


In [4]:
# โหลดข้อมูล
all_files = glob.glob(r"C:\\Users\\HP\\Desktop\\LSTM\\data\\*.csv")

print(f"เจอไฟล์ทั้งหมด {len(all_files)} ไฟล์:")
for f in all_files:
    print("  →", f.split("\\")[-1])  # แสดงแค่ชื่อไฟล์

# ส่วนที่เหลือเหมือนเดิมทุกอย่าง
dfs = []
for f in all_files:
    df = pd.read_csv(f, header=None)
    df.columns = ['Time', '1', '2', '3', '4', '5', '6', '7', '8', 'Annotation']
    dfs.append(df)
data = pd.concat(dfs, ignore_index=True)
print(f"รวมข้อมูลทั้งหมด: {len(data):,} แถว จาก {len(all_files)} คืน")    

เจอไฟล์ทั้งหมด 5 ไฟล์:
  → SN001.csv
  → SN002.csv
  → SN003.csv
  → SN005.csv
  → SN006.csv


C:\Users\HP\AppData\Local\Temp\ipykernel_23972\4099449083.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, header=None)
C:\Users\HP\AppData\Local\Temp\ipykernel_23972\4099449083.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, header=None)
C:\Users\HP\AppData\Local\Temp\ipykernel_23972\4099449083.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, header=None)
C:\Users\HP\AppData\Local\Temp\ipykernel_23972\4099449083.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, header=None)
C:\Users\HP\AppData\Local\Temp\ipykernel_23972\4099449083.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, header=None)


รวมข้อมูลทั้งหมด: 34,500,869 แถว จาก 5 คืน


In [5]:
eeg_cols = ['1', '2', '3', '4']
annotations = data['Annotation'].astype(str).values
eeg_data = data[eeg_cols].values.astype(np.float32)
ch_names = ['F4-M1', 'C4-M1', 'O2-M1', 'C3-M2']
info = mne.create_info(ch_names, sfreq=256, ch_types=['eeg']*4)
raw = mne.io.RawArray(eeg_data.T * 1e-6, info)

raw.set_eeg_reference('average')
raw.filter(0.5, 40)
raw.notch_filter(50)

Creating RawArray with float64 data, n_channels=4, n_times=34500869
    Range : 0 ... 34500868 =      0.000 ... 134769.016 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1691 samples (6.605 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filt

<RawArray | 4 x 34500869 (134769.0 s), ~1.03 GiB, data loaded>

In [8]:
# ------------------- 2. Epoching 30 วินาที (แก้ inconsistent samples 100%) -------------------
# ------------------- 2. Epoching 30 วินาที - Single-Channel Version (ตาม paper เป๊ะ ๆ) -------------------
sfreq = 256
epoch_len = 30 * sfreq  # 7680 samples
n_epochs = len(eeg_data) // epoch_len

X = []  # จะเป็น (n_epochs, 1, 7680)
y = []

label_map = {
    ' Sleep stage W': 0,
    'Sleep stage W': 0,
    ' W': 0,
    'Wake': 0,
    
    ' Sleep stage N1': 1,
    'Sleep stage N1': 1,
    ' N1': 1,
    
    ' Sleep stage N2': 2,
    'Sleep stage N2': 2,
    ' N2': 2,
    
    ' Sleep stage N3': 3,
    'Sleep stage N3': 3,
    ' N3': 3,
    'SWS': 3,
    
    ' Sleep stage R': 4,
    'Sleep stage R': 4,
    ' R': 4,
    'REM': 4
}

print("กำลังสร้าง epochs แบบ single-channel...")
for i in range(n_epochs):
    start = i * epoch_len
    end = start + epoch_len
    
    # ใช้แค่ช่องเดียว (ช่อง '1' = ช่องแรกในข้อมูลคุณ)
    epoch = eeg_data[start:end, 0]  # index 0 = ช่องแรก
    epoch = (epoch - epoch.mean()) / (epoch.std() + 1e-8)  # z-score
    epoch = epoch.reshape(1, -1)  # shape: (1, 7680)
    
    X.append(epoch)
    
    mid = start + epoch_len // 2
    label_str = str(annotations[mid])
    
    if label_str in label_map:
        y.append(label_map[label_str])
    else:
        # ถ้า label ไม่ตรง → ข้าม epoch นี้ (ไม่เพิ่ม X)
        X.pop()  # ลบ epoch ที่เพิ่งเพิ่มไป
        continue

X = np.array(X)  # shape: (n_epochs, 1, 7680)
y = np.array(y)

print(f"สร้าง epoch สำเร็จ: {len(X)} epochs (single-channel)")

print(f"สร้าง epoch เสร็จ: {len(X)} epochs (X และ y ขนาดเท่ากันแล้ว)")
print("Class distribution:", np.bincount(y))

กำลังสร้าง epochs แบบ single-channel...
สร้าง epoch สำเร็จ: 4489 epochs (single-channel)
สร้าง epoch เสร็จ: 4489 epochs (X และ y ขนาดเท่ากันแล้ว)
Class distribution: [ 969  551 1598  712  659]


In [9]:
# ------------------- 3. แบ่งข้อมูล -------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [10]:
# ------------------- 4. Dataset (มี signal shifting 3 แบบ) -------------------
class SingleChannelDataset(Dataset):
    def __init__(self, X, y, shift='none'):
        self.X = torch.FloatTensor(X)  # (N, 1, 7680)
        self.y = torch.LongTensor(y)
        self.shift = shift
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        eeg = self.X[idx]  # (1, 7680)
        
        if self.shift == 'previous':
            shifted = torch.cat([eeg[:, 500:], eeg[:, -500:]], dim=1)
        elif self.shift == 'next':
            shifted = torch.cat([eeg[:, :-500], eeg[:, :500]], dim=1)
        else:
            shifted = eeg
        
        return shifted.squeeze(0), self.y[idx]  # (7680,), label

batch_size = 256  # หรือ 256 ถ้า GPU แรงพอ (แนะนำ 128 ก่อน)

train_current = DataLoader(SingleChannelDataset(X_train, y_train, 'none'), batch_size=batch_size, shuffle=True)
train_previous = DataLoader(SingleChannelDataset(X_train, y_train, 'previous'), batch_size=batch_size, shuffle=True)
train_next = DataLoader(SingleChannelDataset(X_train, y_train, 'next'), batch_size=batch_size, shuffle=True)

val_current = DataLoader(SingleChannelDataset(X_test, y_test, 'none'), batch_size=batch_size)
val_previous = DataLoader(SingleChannelDataset(X_test, y_test, 'previous'), batch_size=batch_size)
val_next = DataLoader(SingleChannelDataset(X_test, y_test, 'next'), batch_size=batch_size)

In [11]:
# ------------------- 5. CNN สำหรับ single-channel -------------------
class SingleChannelCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(1, 10, kernel_size=55, padding=27),
            nn.ReLU(),
            nn.BatchNorm1d(10),
            nn.MaxPool1d(kernel_size=16, stride=8)
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(10, 5, kernel_size=25, padding=12),
            nn.ReLU(),
            nn.BatchNorm1d(5),
            nn.MaxPool1d(kernel_size=8, stride=4)
        )
        self.adaptive_pool = nn.AdaptiveAvgPool1d(240)  # บังคับได้ 240 เสมอ
        self.fc1 = nn.Linear(5 * 240, 10)
        self.fc2 = nn.Linear(10, 5)
    
    def forward(self, x):
        x = x.unsqueeze(1)  # (B, 7680) → (B, 1, 7680)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.adaptive_pool(x)  # (B, 5, 240)
        x = x.view(x.size(0), -1)  # (B, 1200)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# สร้าง 15 CNNs
cnns = [SingleChannelCNN().to(device) for _ in range(15)]

In [12]:
# ------------------- 6. Separating Training with Varying Loss Weights (ตาม paper 100%) -------------------
def train_single_cnn(cnn, train_loader, val_loader, target_class, class_counts, total_samples, epochs=200):
    """
    target_class: 0-4 (Wake, N1, N2, N3, REM)
    class_counts: np.array ของจำนวนแต่ละ class
    total_samples: จำนวน epoch ทั้งหมด
    """
    optimizer = optim.Adam(cnn.parameters(), lr=0.001)
    
    # Varying loss weights ตาม paper
    # 1. Base weight = total / (num_classes * class_count) → inverse proportion
    base_weights = total_samples / (5 * class_counts.astype(float))
    
    # 2. ดัน weight ของ target_class สูงขึ้น (paper ทำเพื่อให้ CNN เชี่ยวชาญ class นั้น)
    weights = base_weights.copy()
    weights[target_class] *= 2.0  # ปรับคูณ 2 (paper ไม่ระบุตัวเลขแน่นอน แต่ทำแบบนี้)
    
    # Normalize ให้ sum = 1 (optional แต่ดี)
    weights = weights / weights.sum()
    
    weights = torch.FloatTensor(weights).to(device)
    print(f"   → Weights for this CNN (class {target_class}): {weights.cpu().numpy().round(3)}")
    
    criterion = nn.CrossEntropyLoss(weight=weights)
    
    best_val_acc = 0
    patience = 0
    
    for epoch in range(epochs):
        cnn.train()
        total_loss = 0
        for x, t in train_loader:
            x, t = x.to(device), t.to(device)
            optimizer.zero_grad()
            out = cnn(x)
            loss = criterion(out, t)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        # Validation every 150 iterations (ตาม paper)
        if epoch % 150 == 0 or epoch == epochs-1:
            cnn.eval()
            preds, trues = [], []
            with torch.no_grad():
                for x, t in val_loader:
                    out = cnn(x.to(device))
                    preds.extend(out.argmax(1).cpu().numpy())
                    trues.extend(t.numpy())
            val_acc = accuracy_score(trues, preds)
            
            print(f"     Validation Acc: {val_acc*100:.2f}%")
            
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                patience = 0
                # torch.save(cnn.state_dict(), f"best_cnn_{target_class}.pth")  # optional
            else:
                patience += 1
                if patience >= 10:
                    print(f"     Early stop at epoch {epoch}")
                    break
    
    return cnn

# ------------------- เรียกใช้ตอนเทรน 15 CNNs -------------------
print("เริ่มเทรน 15 CNNs ด้วย varying loss weights...")

# คำนวณ class counts จาก training set
class_counts = np.bincount(y_train)
total_samples = len(y_train)

for i in range(15):
    set_idx = i // 5  # 0=Previous, 1=Current, 2=Next
    target_class = i % 5
    
    set_name = ['Previous', 'Current', 'Next'][set_idx]
    class_name = ['Wake', 'N1', 'N2', 'N3', 'REM'][target_class]
    
    print(f"\nเทรน CNN {i+1}/15: {set_name}-{class_name}")
    
    if set_idx == 0:
        loader = train_previous
        val_loader = val_previous
    elif set_idx == 1:
        loader = train_current
        val_loader = val_current
    else:
        loader = train_next
        val_loader = val_next
    
    cnns[i] = train_single_cnn(cnns[i], loader, val_loader, target_class, class_counts, total_samples)

เริ่มเทรน 15 CNNs ด้วย varying loss weights...

เทรน CNN 1/15: Previous-Wake
   → Weights for this CNN (class 0): [0.278 0.244 0.084 0.189 0.204]
     Validation Acc: 22.83%
     Validation Acc: 66.48%
     Validation Acc: 66.15%

เทรน CNN 2/15: Previous-N1
   → Weights for this CNN (class 1): [0.126 0.442 0.076 0.171 0.185]
     Validation Acc: 23.61%
     Validation Acc: 65.48%
     Validation Acc: 64.37%

เทรน CNN 3/15: Previous-N2
   → Weights for this CNN (class 2): [0.147 0.258 0.178 0.2   0.216]
     Validation Acc: 37.86%
     Validation Acc: 68.26%
     Validation Acc: 67.04%

เทรน CNN 4/15: Previous-N3
   → Weights for this CNN (class 3): [0.132 0.233 0.08  0.36  0.195]
     Validation Acc: 18.26%
     Validation Acc: 66.04%
     Validation Acc: 67.26%

เทรน CNN 5/15: Previous-REM
   → Weights for this CNN (class 4): [0.13  0.229 0.079 0.177 0.384]
     Validation Acc: 23.39%
     Validation Acc: 66.48%
     Validation Acc: 67.93%

เทรน CNN 6/15: Current-Wake
   → Weights for

In [23]:
import os

# สร้างโฟลเดอร์เก็บโมเดล
MODEL_DIR = "zleep_cnns_models"
os.makedirs(MODEL_DIR, exist_ok=True)

print("กำลังบันทึก 15 CNNs...")
for i in range(15):
    path = os.path.join(MODEL_DIR, f"cnn_{i}.pth")
    torch.save(cnns[i].state_dict(), path)
    print(f"  บันทึก CNN {i+1}/15 → {path}")

print("บันทึก CNNs ทั้งหมดเสร็จแล้ว! ครั้งต่อไปโหลดใช้ได้เลย")

กำลังบันทึก 15 CNNs...
  บันทึก CNN 1/15 → zleep_cnns_models\cnn_0.pth
  บันทึก CNN 2/15 → zleep_cnns_models\cnn_1.pth
  บันทึก CNN 3/15 → zleep_cnns_models\cnn_2.pth
  บันทึก CNN 4/15 → zleep_cnns_models\cnn_3.pth
  บันทึก CNN 5/15 → zleep_cnns_models\cnn_4.pth
  บันทึก CNN 6/15 → zleep_cnns_models\cnn_5.pth
  บันทึก CNN 7/15 → zleep_cnns_models\cnn_6.pth
  บันทึก CNN 8/15 → zleep_cnns_models\cnn_7.pth
  บันทึก CNN 9/15 → zleep_cnns_models\cnn_8.pth
  บันทึก CNN 10/15 → zleep_cnns_models\cnn_9.pth
  บันทึก CNN 11/15 → zleep_cnns_models\cnn_10.pth
  บันทึก CNN 12/15 → zleep_cnns_models\cnn_11.pth
  บันทึก CNN 13/15 → zleep_cnns_models\cnn_12.pth
  บันทึก CNN 14/15 → zleep_cnns_models\cnn_13.pth
  บันทึก CNN 15/15 → zleep_cnns_models\cnn_14.pth
บันทึก CNNs ทั้งหมดเสร็จแล้ว! ครั้งต่อไปโหลดใช้ได้เลย


In [ ]:
#โค้ดโหลดกลับมาใช้ (วางก่อนเทรน BiLSTM หรือตอนเริ่มโค้ดใหม่)
# ตรวจสอบและโหลด CNNs ที่บันทึกไว้
if os.path.exists(MODEL_DIR) and len(os.listdir(MODEL_DIR)) == 15:
    print("พบโมเดล CNN ที่บันทึกไว้ → โหลดแทนการเทรนใหม่")
    cnns = []
    for i in range(15):
        cnn = SingleChannelCNN().to(device)
        cnn.load_state_dict(torch.load(os.path.join(MODEL_DIR, f"cnn_{i}.pth"), map_location=device))
        cnn.eval()  # ใช้ใน no_grad ตอน extract features
        cnns.append(cnn)
    print("โหลด CNNs ครบ 15 ตัวเรียบร้อย!")
else:
    print("ไม่พบโมเดลที่บันทึก → ต้องเทรน CNNs ใหม่")
    # ... (โค้ดเทรน CNNs เดิมของคุณ)

พบโมเดล CNN ที่บันทึกไว้ → โหลดแทนการเทรนใหม่
โหลด CNNs ครบ 15 ตัวเรียบร้อย!


In [22]:
# ------------------- 7. Extract Features + BiLSTM (แก้ shifted แล้ว) -------------------

def extract_features_single(X_batch):
    """
    X_batch: (B, 7680) หรือ (B, 1, 7680) หรืออะไรก็ตาม
    ทำให้เป็น (B, 1, 7680) เสมอ ก่อนส่งเข้า CNN
    """
    features = []
    
    # บังคับ shape ให้เป็น (B, 7680) ก่อน
    x = X_batch.clone()
    if x.dim() == 3:
        x = x.squeeze(1)  # (B, 1, 7680) → (B, 7680)
    elif x.dim() == 4:
        x = x.squeeze(1).squeeze(1)  # ลบมิติพิเศษทั้งหมด
    
    # เพิ่ม channel dim กลับ
    x = x.unsqueeze(1)  # (B, 7680) → (B, 1, 7680)
    
    with torch.no_grad():
        for i, cnn in enumerate(cnns):
            if i // 5 == 0:  # Previous
                shifted = torch.cat([x[:, :, 500:], x[:, :, -500:]], dim=2)
            elif i // 5 == 2:  # Next
                shifted = torch.cat([x[:, :, :-500], x[:, :, :500]], dim=2)
            else:  # Current
                shifted = x
            
            out = cnn(shifted.to(device))
            features.append(out)
    
    cat_features = torch.cat(features, dim=1)  # (B, 1200)
    
    # พิมพ์ขนาดแค่ครั้งแรก
    if not hasattr(extract_features_single, "printed"):
        print(f"Feature size ที่ได้จริง: {cat_features.shape[1]}")
        extract_features_single.printed = True
    
    return cat_features

# ------------------- หา feature_size จาก dummy batch (ก่อนสร้าง BiLSTM) -------------------
print("กำลังคำนวณ feature size จาก dummy batch...")

# ใช้ batch เล็ก ๆ จาก train set เพื่อดูขนาด features จริง
dummy_batch = torch.FloatTensor(X_train[:32])
if dummy_batch.dim() == 4:
    dummy_batch = dummy_batch.squeeze(1)  # (32, 1, 7680) → (32, 7680)

dummy_features = extract_features_single(dummy_batch)
feature_size = dummy_features.shape[1]

print(f"Feature size ที่ได้จริง = {feature_size} (ใช้ค่านี้ใน BiLSTM)")

bilstm = BiLSTM(input_size=feature_size).to(device)  # feature_size จาก print ก่อนหน้า
optimizer = optim.Adam(bilstm.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()



batch_size = 128

print("เริ่มเทรน BiLSTM...")
for epoch in range(1, 201):
    bilstm.train()
    total_loss = 0
    for i in range(0, len(X_train), batch_size):
        end = min(i + batch_size, len(X_train))
        
        batch_X = torch.FloatTensor(X_train[i:end])
        # ถ้ามี channel dim → ลบออก
        if batch_X.dim() == 3:
            batch_X = batch_X.squeeze(1)  # (B, 1, 7680) → (B, 7680)
        
        batch_y = torch.LongTensor(y_train[i:end])
        
        features = extract_features_single(batch_X)
        optimizer.zero_grad()
        out = bilstm(features.unsqueeze(1))  # (B, 1, 1200)
        loss = criterion(out, batch_y.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    if epoch % 10 == 0:
        bilstm.eval()
        with torch.no_grad():
            val_X = torch.FloatTensor(X_test)
            if val_X.dim() == 3:
                val_X = val_X.squeeze(1)
            val_features = extract_features_single(val_X)
            val_out = bilstm(val_features.unsqueeze(1))
            val_pred = val_out.argmax(1).cpu().numpy()
            val_acc = accuracy_score(y_test, val_pred)
            print(f"Epoch {epoch:3d}/200 | Loss: {total_loss/(len(X_train)//batch_size + 1):.4f} | Val Acc: {val_acc*100:5.2f}%")

กำลังคำนวณ feature size จาก dummy batch...


RuntimeError: Expected 2D (unbatched) or 3D (batched) input to conv1d, but got input of size: [32, 1, 1, 7680]

In [ ]:
# ------------------- 8. Final Test -------------------
bilstm.eval()
with torch.no_grad():
    test_features = extract_features_single(torch.FloatTensor(X_test).squeeze(1))
    test_out = bilstm(test_features.unsqueeze(1))
    pred = test_out.argmax(1).cpu().numpy()

acc = accuracy_score(y_test, pred)
print(f"\nZleepAnlystNet (Single-Channel) FINAL ACCURACY: {acc*100:.2f}%")
print(classification_report(y_test, pred, target_names=['Wake','N1','N2','N3','REM']))

plt.figure(figsize=(8,6))
sns.heatmap(confusion_matrix(y_test, pred), annot=True, fmt='d', cmap='Blues',
            xticklabels=['Wake','N1','N2','N3','REM'])
plt.title(f"ZleepAnlystNet Single-Channel - Accuracy: {acc*100:.2f}%")
plt.show()